In [1]:
from PIL import Image
from PIL import ImageFilter
import numpy as np
from scipy.sparse import csr_matrix
from scipy.ndimage.morphology import binary_dilation
from scipy.ndimage import generate_binary_structure
from scipy.misc import imsave

In [2]:
def get_rand_att_from_edge(edge_obj, num_edge_points_max):
    edge_arr = np.array(edge_obj, np.uint8)
    true_indices = np.nonzero(edge_arr)
    num_true = true_indices[0].shape[0]
    num_rand_shape = np.random.randint(0,num_edge_points_max+1)
    rand_indices = np.random.choice(num_true, num_rand_shape)
    if rand_indices.size != 0:
        data = np.ones(rand_indices.size, np.uint8)
        row_ind = []
        col_ind = []
        for idx in rand_indices:
            row_ind.append(true_indices[0][idx])
            col_ind.append(true_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=edge_arr.shape)
        sparse_arr = sparse_mat.toarray().astype(np.uint8)
        struct1 = generate_binary_structure(2, 2)
        size_att = np.random.randint(9, 36)
        rand_shape_arr = binary_dilation(sparse_arr, structure=struct1, iterations=size_att).astype(sparse_arr.dtype)
    else:
        rand_shape_arr = np.zeros(edge_arr.shape, np.uint8)
        print('No false or rand att.')
    return rand_shape_arr

In [45]:
def get_prob_map(seg, att):
    '''
    :param seg: [h, w, 1], np.int32
    :param att: [h, w, 1], np.int32
    :return: prob_map, [h, w, 1], np.float32
    '''

    seg = np.squeeze(seg)
    att = np.squeeze(att)
    # get label map, 0 for background, 1 for negative, 2 for obj
    label_map = seg + att

    # generate confidence values in obj area, mostly >= 0.9, a few < 0.1 by choosing random seed an dilate
    obj_bool = np.greater(label_map, 1)
    total_indices = np.nonzero(obj_bool)
    num_total_indices = total_indices[0].shape[0]
    num_low_seed = np.random.randint(0,5)  # number of seed pixels with low confidence
    print('num low seed: {}'.format(num_low_seed))
    if num_low_seed != 0:
        low_prob_indices = np.random.choice(num_total_indices, num_low_seed)
        data = np.ones(num_low_seed, dtype=np.int32)
        row_ind = []
        col_ind = []
        for idx in low_prob_indices:
            row_ind.append(total_indices[0][idx])
            col_ind.append(total_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=att.shape, dtype=np.int32)
        # get dense arr and dilate
        dense_arr_low_obj = sparse_mat.toarray().astype(np.int32)
        struct1 = generate_binary_structure(2, 2)
        rand_iter = np.random.randint(0,30)
        low_prob_obj_area = binary_dilation(dense_arr_low_obj, structure=struct1, iterations=rand_iter).astype(np.int32)
        low_prob_obj_area = np.multiply(low_prob_obj_area, seg)
        high_prob_obj_area = seg - low_prob_obj_area
        # get indices of the low area and multiply by random float [0.0, 0.5]
        low_prob_obj_area_indices = np.nonzero(low_prob_obj_area)
        num_low_prob_obj_area = low_prob_obj_area_indices[0].shape[0]
        data = np.float32(np.random.randint(0, 10, num_low_prob_obj_area)) / 100.0
        row_ind = []
        col_ind = []
        for idx in range(num_low_prob_obj_area):
            row_ind.append(low_prob_obj_area_indices[0][idx])
            col_ind.append(low_prob_obj_area_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=att.shape, dtype=np.float32)
        low_prob_obj_part = sparse_mat.toarray().astype(np.float32)
        # get indices of the high area and multiply by random float [0.95, 1.0]
        high_prob_obj_area_indices = np.nonzero(high_prob_obj_area)
        num_high_prob_obj_area = high_prob_obj_area_indices[0].shape[0]
        data = np.float32(np.random.randint(90, 100, num_high_prob_obj_area)) / 100.0
        row_ind = []
        col_ind = []
        for idx in range(num_high_prob_obj_area):
            row_ind.append(high_prob_obj_area_indices[0][idx])
            col_ind.append(high_prob_obj_area_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=att.shape, dtype=np.float32)
        high_prob_obj_part = sparse_mat.toarray().astype(np.float32)
        obj_prob = low_prob_obj_part + high_prob_obj_part
    else:
        data = np.float32(np.random.randint(90, 100, num_total_indices)) / 100.0
        row_ind = []
        col_ind = []
        for idx in range(num_total_indices):
            row_ind.append(total_indices[0][idx])
            col_ind.append(total_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=att.shape, dtype=np.float32)
        obj_prob = sparse_mat.toarray().astype(np.float32)
        
     # generate confidence values in negative area, mostly < 0.1, a few >= 0.9 by choosing random seed an dilate
    nega_bool = np.equal(label_map, 1)
    nega_arr = nega_bool.astype(np.int32)
    total_indices = np.nonzero(nega_bool)
    num_total_indices = total_indices[0].shape[0]
    num_high_seed = np.random.randint(0, 5)  # number of seed pixels with high confidence
    if num_high_seed != 0:
        high_prob_indices = np.random.choice(num_total_indices, num_high_seed)
        data = np.ones(num_high_seed, dtype=np.int32)
        row_ind = []
        col_ind = []
        for idx in high_prob_indices:
            row_ind.append(total_indices[0][idx])
            col_ind.append(total_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=att.shape, dtype=np.int32)
        # get dense arr and dilate
        dense_arr_high_obj = sparse_mat.toarray().astype(np.int32)
        struct1 = generate_binary_structure(2, 2)
        rand_iter = np.random.randint(0,30)
        high_prob_obj_area = binary_dilation(dense_arr_high_obj, structure=struct1, iterations=rand_iter).astype(np.int32)
        high_prob_obj_area = np.multiply(high_prob_obj_area, nega_arr)
        low_prob_obj_area = nega_arr - high_prob_obj_area
        # get indices of the high area and multiply by random float [0.9, 1.0]
        high_prob_obj_area_indices = np.nonzero(high_prob_obj_area)
        num_high_prob_obj_area = high_prob_obj_area_indices[0].shape[0]
        data = np.float32(np.random.randint(90, 100, num_high_prob_obj_area)) / 100.0
        row_ind = []
        col_ind = []
        for idx in range(num_high_prob_obj_area):
            row_ind.append(high_prob_obj_area_indices[0][idx])
            col_ind.append(high_prob_obj_area_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=att.shape, dtype=np.float32)
        high_prob_obj_part = sparse_mat.toarray().astype(np.float32)
        # get indices of the low area and multiply by random float [0.0, 0.1]
        low_prob_obj_area_indices = np.nonzero(low_prob_obj_area)
        num_low_prob_obj_area = low_prob_obj_area_indices[0].shape[0]
        data = np.float32(np.random.randint(0, 10, num_low_prob_obj_area)) / 100.0
        row_ind = []
        col_ind = []
        for idx in range(num_low_prob_obj_area):
            row_ind.append(low_prob_obj_area_indices[0][idx])
            col_ind.append(low_prob_obj_area_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=att.shape, dtype=np.float32)
        low_prob_obj_part = sparse_mat.toarray().astype(np.float32)
        nega_prob = low_prob_obj_part + high_prob_obj_part
    else:
        data = np.float32(np.random.randint(0, 10, num_total_indices)) / 100.0
        row_ind = []
        col_ind = []
        for idx in range(num_total_indices):
            row_ind.append(total_indices[0][idx])
            col_ind.append(total_indices[1][idx])
        sparse_mat = csr_matrix((data, (row_ind, col_ind)), shape=att.shape, dtype=np.float32)
        nega_prob = sparse_mat.toarray().astype(np.float32)

    full_prob_map = obj_prob + nega_prob
    return full_prob_map

In [46]:
seg_file = 'seg1.png'
seg_arr = np.array(Image.open(seg_file), np.uint8)
seg_arr_bool = np.greater(seg_arr, 0)
seg_arr = seg_arr_bool.astype(np.uint8)
imsave('normal_seg.png', seg_arr.astype(np.float32))
struct1 = generate_binary_structure(2, 2)
att_arr = binary_dilation(seg_arr, structure=struct1, iterations=30).astype(seg_arr.dtype)
# Compute random shape
att_obj = Image.fromarray(att_arr, mode='L')
edge_obj = att_obj.filter(ImageFilter.FIND_EDGES)
rand_shape_arr = get_rand_att_from_edge(edge_obj, 5)
# Compute close by random false attention
large_dilate = binary_dilation(att_arr, structure=struct1, iterations=40).astype(att_arr.dtype)
large_dilate_obj = Image.fromarray(large_dilate)
large_edge_obj = large_dilate_obj.filter(ImageFilter.FIND_EDGES)
false_att_arr = get_rand_att_from_edge(large_edge_obj, 3)
att_rand = att_arr + false_att_arr + rand_shape_arr
att_rand_bool = np.greater(att_rand, 0)
att_rand = att_rand_bool.astype(np.uint8)
imsave('normal_att.png', att_rand.astype(np.float32))
prob_map = get_prob_map(seg_arr, att_rand)
imsave('prob_map.png', prob_map)

num low seed: 5
